In [ ]:
import numpy as np
import pandas as pd
from textwrap import wrap
from matplotlib import pyplot as plt
from sklearn import metrics
import matplotlib.ticker as mticker
import sys
from hsbmpy import plot_topic_size, get_max_available_L

In [ ]:
directory="/home/fvalle/phd/datasets/tcga/TCGA/"

In [ ]:
algorithm = 'lda'
L = get_max_available_L(directory, algorithm=algorithm)

# topic size

In [ ]:
for l in range(0,L+1):
    plot_topic_size(directory,l, algorithm=algorithm)

# Geneontology

In [ ]:
from geneontology import get_ontology_df, ensg_to_symbol
from tableanalyser import get_symbol

In [ ]:
l=L
df_topics = pd.read_csv("%s/%s/%s_level_%d_topics.csv"%(directory,algorithm,algorithm,l))

In [ ]:
#import gseapy as gs
#gs.get_library_name()

In [ ]:
back_sea=[]
for topic in df_topics.columns:
    for g in df_topics[topic].dropna().values:
        try:
            back_sea.append(get_symbol(g))
        except:
            print("error %s"%g)

In [ ]:
for itopic,topic in enumerate(df_topics.columns):
    try:
        enriched_topic = pd.read_csv("%s/%s/gsea/gsea_level_%d_topic%d.csv"%(directory,algorithm,l,itopic), index_col=[0])
        print(topic)
    except:
        try:
            symbols = ensg_to_symbol(df_topics.loc[:,topic].dropna().values)
            print(topic)
            enriched_topic = get_ontology_df(symbols, background=back_sea).sort_values(by=['Adjusted P-value'], ascending=True)
            enriched_topic = enriched_topic.loc[enriched_topic.index.values[:20],:]
            enriched_topic.to_csv("%s/%s/gsea/gsea_level_%d_topic%d.csv"%(directory,algorithm,l,itopic))
        except:
            print(*sys.exc_info())
    print(enriched_topic)

In [ ]:
topic_pvalues = []
topic_gos = []
for itopic,topic in enumerate(df_topics.columns):
    try:
        enriched_topic = pd.read_csv("%s/gsea/gsea_level_%d_topic%d.csv"%(directory,l,itopic))
        if len(enriched_topic.index) >0:
            p_val = np.sort(enriched_topic['Adjusted P-value'])[0]
            topic_pvalues.append(-np.log10(p_val))
            for goc in enriched_topic['Gene_set'][:10].unique():
                topic_gos.append(goc)
        print(topic)
    except:
        print("error", sys.exc_info()[0])

In [ ]:
fig = plt.figure()
x = np.arange(1,1+len(topic_pvalues))
c, _, _ = plt.hist(topic_pvalues, histtype='step', lw=2)
plt.plot([-np.log10(0.05) for _ in np.linspace(1,10,num=10)],np.arange(0,np.max(c)+5,(np.max(c)+5)/10), ls='--', lw=5, label="$\\alpha=0.05$")
plt.xlabel('-log(P-value)', fontsize=16)
plt.ylabel("number of topics")
#plt.ylim(0,0.055)
#plt.yscale('log')
plt.legend(fontsize=16)
fig.savefig("%s/pvaluescrosstopic(%d).png"%(directory,l))

In [ ]:
fig = plt.figure(figsize=(20,10))
gos, goscounts = np.unique(topic_gos,return_counts=True)
plt.barh(["\n".join(wrap(str(l).replace('_',' '),20)) for l in gos], goscounts)
plt.yticks(fontsize=15)
plt.show()
fig.savefig("%s/pvaluecategories(%d).pdf"%(directory,l))